In [1]:
import numpy as np 
import pandas as pd 
import statsmodels.tsa.stattools
import statsmodels.graphics.tsaplots
import pickle 
import sys
import time 
import warnings
from datetime import datetime, timedelta 

In [2]:
DATA_PATH = '../data/'
warnings.filterwarnings('ignore')

# Stock return and covariance

In [12]:
stock_df = pd.read_csv(DATA_PATH + 'totalPrice.csv').rename(columns={'Unnamed: 0': 'Code'}).set_index('Code')
stock_df.head()

,2017-01-02 00:00:00,2017-01-03 00:00:00,2017-01-04 00:00:00,2017-01-05 00:00:00,2017-01-06 00:00:00,2017-01-09 00:00:00,2017-01-10 00:00:00,2017-01-11 00:00:00,2017-01-12 00:00:00,2017-01-13 00:00:00,...,2018-12-19 00:00:00,2018-12-20 00:00:00,2018-12-21 00:00:00,2018-12-24 00:00:00,2018-12-25 00:00:00,2018-12-26 00:00:00,2018-12-27 00:00:00,2018-12-28 00:00:00,2018-12-31 00:00:00,2019-01-01 00:00:00
Code,,,,,,,,,,,,,,,,,,,,,
AAPL UW Equity,115.82,116.15,116.02,116.61,117.91,118.99,119.11,119.75,119.25,119.04,...,160.89,156.83,150.73,146.83,146.83,157.17,156.15,156.23,157.74,157.74
AXP UN Equity,74.08,75.35,76.26,75.32,75.47,75.86,76.65,76.91,76.88,76.62,...,98.77,95.77,91.33,89.50,89.50,93.84,94.68,94.42,95.32,95.32
BA UN Equity,155.68,156.97,158.62,158.71,159.10,158.32,159.07,159.40,158.29,158.83,...,319.55,313.05,304.55,294.16,294.16,313.93,317.14,316.38,322.50,322.50
CAT UN Equity,92.74,93.99,93.57,93.00,93.04,92.37,93.83,94.65,93.99,94.48,...,122.33,121.55,120.07,116.95,116.95,124.76,126.67,125.61,127.07,127.07
CSCO UW Equity,30.22,30.54,30.10,30.17,30.23,30.18,30.38,30.15,30.04,30.07,...,43.14,42.49,41.85,40.28,40.28,42.47,42.91,42.77,43.33,43.33


In [13]:
asset = stock_df.values
A = asset[:, 0:-1]
A_plus = asset[:, 1:]
returns_matrix = (A_plus - A) / A
sample_mean_returns = np.mean(returns_matrix, axis=1)
sample_covariance_matrix = np.cov(returns_matrix)
# np.save('covariance_matrix.npy',sample_covariance_matrix)

In [14]:
# output stock_name_arr 
stock_name_arr = stock_df.index.to_numpy()
print('Stock name array looks like:\n', stock_name_arr)
with open(DATA_PATH + 'stock_name_arr.pkl', 'wb') as file:
    pickle.dump(stock_name_arr, file)

# output time
stock_time_arr = stock_df.T.index.to_numpy()
print('Stock time spam array looks like:\n', stock_time_arr)
with open(DATA_PATH + 'stock_time_arr.pkl', 'wb') as file:
    pickle.dump(stock_time_arr, file)
    
# output stock covariance
print('Covariance matrix looks like:\n', sample_covariance_matrix)
with open(DATA_PATH + 'cov_mat.pkl', 'wb') as file:
    pickle.dump(sample_covariance_matrix, file)

Stock name array looks like:
 ['AAPL UW Equity' 'AXP UN Equity' 'BA UN Equity' ... '192080 KS Equity'
 '192820 KS Equity' '316140 KS Equity']
Stock time spam array looks like:
 ['2017-01-02 00:00:00' '2017-01-03 00:00:00' '2017-01-04 00:00:00'
 '2017-01-05 00:00:00' '2017-01-06 00:00:00' '2017-01-09 00:00:00'
 '2017-01-10 00:00:00' '2017-01-11 00:00:00' '2017-01-12 00:00:00'
 '2017-01-13 00:00:00' '2017-01-16 00:00:00' '2017-01-17 00:00:00'
 '2017-01-18 00:00:00' '2017-01-19 00:00:00' '2017-01-20 00:00:00'
 '2017-01-23 00:00:00' '2017-01-24 00:00:00' '2017-01-25 00:00:00'
 '2017-01-26 00:00:00' '2017-01-27 00:00:00' '2017-01-30 00:00:00'
 '2017-01-31 00:00:00' '2017-02-01 00:00:00' '2017-02-02 00:00:00'
 '2017-02-03 00:00:00' '2017-02-06 00:00:00' '2017-02-07 00:00:00'
 '2017-02-08 00:00:00' '2017-02-09 00:00:00' '2017-02-10 00:00:00'
 '2017-02-13 00:00:00' '2017-02-14 00:00:00' '2017-02-15 00:00:00'
 '2017-02-16 00:00:00' '2017-02-17 00:00:00' '2017-02-20 00:00:00'
 '2017-02-21 00:00:

# Sector and Market Info

In [15]:
sector_df = pd.read_csv(DATA_PATH + 'sector.csv').rename(columns={'Unnamed: 0': 'Code', 'INDUSTRY_SECTOR': 'Sector'})

In [16]:
sector_df['Market'] = sector_df.apply(lambda x: x.Code.split(' ')[1], axis=1)

## Market

In [17]:
sector_df.Market.value_counts()

UN    740
UW    337
JT    225
CH    162
LN    128
KS     97
HK     49
SM     35
IM     35
MF     33
GY     29
CC     29
SW     25
SE     19
AV      6
UQ      5
GA      2
UR      2
UA      2
Name: Market, dtype: int64

In [18]:
market_dict = {}
market_arr = sector_df.Market.unique()
for key in market_arr:
    market_dict[key] = sector_df[sector_df.Market == key].Code.to_numpy()
    
with open(DATA_PATH + 'market_dict.pkl', 'wb') as file:
    pickle.dump(market_dict, file)

## Sector

In [19]:
sector_df.Sector.value_counts()

Financial                 439
Consumer, Non-cyclical    342
Industrial                308
Consumer, Cyclical        283
Technology                139
Communications            139
Basic Materials           127
Utilities                  88
Energy                     87
Diversified                 8
Name: Sector, dtype: int64

In [20]:
sector_dict = {}
sector_arr = sector_df.Sector.unique()
for key in sector_arr:
    sector_dict[key] = sector_df[sector_df.Sector == key].Code.to_numpy()
    
with open(DATA_PATH + 'sector_dict.pkl', 'wb') as file:
    pickle.dump(sector_dict, file)

# ARIMA

In [22]:
#create a series for the 1-lag difference
def draw_acf_pacf(ts, lags=31):
    f = plt.figure(facecolor='white')
    ax1 = f.add_subplot(211)
    statsmodels.graphics.tsaplots.plot_acf(ts, lags=31, ax=ax1)
    ax2 = f.add_subplot(212)
    statsmodels.graphics.tsaplots.plot_pacf(ts, lags=31, ax=ax2)
    plt.show()

def testStationarity(time_series):
    dftest = statsmodels.tsa.stattools.adfuller(time_series)
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    return dfoutput


def proper_model(data_ts, maxLag_p = 5, maxLag_q = 5):
    init_aic = sys.maxsize
    init_p = 0
    init_q = 0
    init_properModel = None
    for p in np.arange(maxLag_p):
        for q in np.arange(maxLag_q):
            try:
                model = statsmodels.tsa.arima_model.ARMA(data_ts, order=(p, q), freq = 'D')
                results_ARMA = model.fit(disp=-1, method='css')
                aic = results_ARMA.aic
                if aic < init_aic:
                    init_p = p
                    init_q = q
                    init_properModel = results_ARMA
                    init_aic = aic
            except:
                continue
            
    return init_properModel, init_p, init_q

def diff_to_stationary(ts):
    if(testStationarity(ts)['p-value'] <= 0.05):
        return ts,0
    else:
        ts_diff = ts.diff(1).dropna()
        num = 1
        while(testStationarity(ts_diff)['p-value'] > 0.05):
            ts_diff = ts_diff.diff(1).dropna()
            num += 1
        return ts_diff, num

In [ ]:
stock_df = pd.read_csv(DATA_PATH + 'totalPrice.csv').rename(columns={'Unnamed: 0': 'Code'}).set_index('Code')

stock_df = stock_df.T 
stock_df.index = pd.to_datetime(stock_df.index)
def arima_main(df):
    cols = df.columns
    df = df.reset_index(drop=False)
    df.rename(columns={'index':'DATE'}, inplace = True)
    df['DATE'] = pd.to_datetime(df['DATE'])
    
    def arima_predict(ric, training_days=30):
        start = time.time()
        close_price = df[['DATE', ric]]
        helper = pd.DataFrame({'DATE': pd.date_range(close_price['DATE'].min(), close_price['DATE'].max())})
        close_price = pd.merge(close_price, helper, on='DATE', how='outer').sort_values('DATE')
        close_price[ric] = close_price[ric].interpolate(method='linear')   
        close_price.set_index(pd.to_datetime(close_price.DATE), inplace=True) # set the index to be the DATE
        close_price.sort_index(inplace=True)  # sort the dataframe by the newly created datetime index


        last_date = close_price.index.to_list()[-1] - timedelta(days=training_days)
        close_price = close_price[close_price.index >= last_date] 
        ts = close_price[ric]
        ts.index = pd.to_datetime(close_price.index)

        ts_diff, num_of_diff = diff_to_stationary(ts)

        inf_lst = proper_model(ts_diff)
        
        current = close_price.index.to_list()[-1]
        end_time = current
        day1 = current + timedelta(days = 1)
        day2 = current + timedelta(days = 2)
        day3 = current + timedelta(days = 3)
        day4 = current + timedelta(days = 4)
        day5 = current + timedelta(days = 5)

        try:
            predict_ts = inf_lst[0].predict(day1, day5, dynamic=True)
        except AttributeError:
            print('No appropriate model')
            return -1

        for i in range(num_of_diff):
            if(num_of_diff - i - 1 != 0):
                predict_ts[day1] = predict_ts[day1] + ts.diff(num_of_diff - i)[end_time]
            else:
                predict_ts[day1] = predict_ts[day1] + ts[end_time]
            predict_ts[day2] = predict_ts[day2] + predict_ts[day1]
            predict_ts[day3] = predict_ts[day3] + predict_ts[day2]
            predict_ts[day4] = predict_ts[day4] + predict_ts[day3]
            predict_ts[day5] = predict_ts[day5] + predict_ts[day4]
        return (predict_ts[day5] - ts[end_time]) / ts[end_time]
    
    pred_dict = {}
    for col in cols:
        start = time.time()
        pred_dict[col] = arima_predict(col)
        print('{} costs time {}'.format(col, time.time() - start))
        
    return pred_dict
pred_dict = arima_main(stock_df)

AAPL UW Equity costs time 0.7318191528320312
AXP UN Equity costs time 1.7196390628814697
BA UN Equity costs time 0.7277822494506836
CAT UN Equity costs time 0.994671106338501
CSCO UW Equity costs time 0.769460916519165
CVX UN Equity costs time 1.2243051528930664
DIS UN Equity costs time 0.7353219985961914
GS UN Equity costs time 0.8185510635375977
HD UN Equity costs time 1.1192269325256348
IBM UN Equity costs time 0.6969869136810303
INTC UW Equity costs time 1.464797019958496
JNJ UN Equity costs time 1.058166742324829
JPM UN Equity costs time 0.7650890350341797
KO UN Equity costs time 1.772650957107544
MCD UN Equity costs time 0.7479281425476074
MMM UN Equity costs time 1.053481101989746
MRK UN Equity costs time 1.1916780471801758
MSFT UW Equity costs time 0.9146931171417236
NKE UN Equity costs time 0.8774452209472656
PFE UN Equity costs time 1.5888688564300537
PG UN Equity costs time 1.600752830505371
TRV UN Equity costs time 1.7596251964569092
UNH UN Equity costs time 1.5893039703369